# QiCore Core - Infrastructure Services

**Interactive tutorial for practical infrastructure tools built on @qi/base**

## What You'll Learn

- ✅ **Configuration management** - Multi-source config loading with validation
- ✅ **Structured logging** - Context accumulation and event-driven logging
- ✅ **Caching strategies** - Memory and Redis backends with unified API
- ✅ **Service integration** - How config, logger, and cache work together
- ✅ **Production patterns** - Real-world examples from working applications

---

## 📚 Prerequisites

This tutorial builds on **[01-qi-base.ipynb](./01-qi-base.ipynb)**. Make sure you understand Result<T> patterns before continuing.

Let's start by importing everything we need:

In [ ]:
// Import Result<T> fundamentals from @qi/base
import { 
  success, failure, match, map, flatMap,
  type Result, type QiError,
  validationError, systemError, configurationError
} from '@qi/base'

// Import core infrastructure services from @qi/core
import { ConfigBuilder } from '@qi/core/config'
import { createLogger } from '@qi/core/logger' 
import { createCache } from '@qi/core/cache'

// Import types for better TypeScript experience
import type { Config, ValidatedConfig } from '@qi/core/config'
import type { Logger } from '@qi/core/logger'
import type { Cache } from '@qi/core/cache'

// Display welcome message
Deno.jupyter.html`
<div style="background: linear-gradient(135deg, #4f46e5 0%, #7c3aed 100%); color: white; padding: 20px; border-radius: 10px; margin: 15px 0;">
  <h3 style="margin: 0 0 15px 0;">🏗️ QiCore Core Tutorial Started!</h3>
  <p style="margin: 0;">Ready to explore configuration, logging, and caching services built on Result&lt;T&gt; patterns!</p>
</div>
`

console.log('✅ @qi/core imports successful!')
console.log('✅ Ready to explore infrastructure services!')

## 1. Configuration Management - Multi-Source Loading

Real applications need configuration from multiple sources: files, environment variables, defaults. QiCore's ConfigBuilder makes this easy:

In [ ]:
// Let's create some sample configuration data
console.log('=== Configuration Management Demo ===\n')

// Method 1: From JavaScript objects (great for defaults)
const defaultConfig = ConfigBuilder.fromObject({
  app: {
    name: 'MyApp',
    port: 3000,
    debug: false
  },
  database: {
    host: 'localhost',
    port: 5432,
    name: 'myapp_dev'
  },
  cache: {
    backend: 'memory',
    maxSize: 1000
  }
})

console.log('1. Default configuration loaded:')
const defaultResult = defaultConfig.build()
match(
  (config) => {
    console.log('   App name:', config.get('app.name'))
    console.log('   Database host:', config.get('database.host'))
    console.log('   Cache backend:', config.get('cache.backend'))
  },
  (error) => console.error('   Failed:', error.message),
  defaultResult
)

console.log()

// Method 2: From environment variables (common in production)
// Set some environment variables for this demo
Deno.env.set('MYAPP_APP_PORT', '8080')
Deno.env.set('MYAPP_APP_DEBUG', 'true')
Deno.env.set('MYAPP_DATABASE_HOST', 'prod-db.example.com')
Deno.env.set('MYAPP_CACHE_BACKEND', 'redis')

const envConfig = ConfigBuilder.fromEnv('MYAPP_')

console.log('2. Environment configuration:')
const envResult = envConfig.build()
match(
  (config) => {
    console.log('   App port (from env):', config.get('app.port'))
    console.log('   Debug mode (from env):', config.get('app.debug'))
    console.log('   Database host (from env):', config.get('database.host'))
  },
  (error) => console.error('   Failed:', error.message),
  envResult
)

console.log()

// Method 3: Merging configurations (environment overrides defaults)
const mergedConfig = defaultConfig.merge(envConfig)

console.log('3. Merged configuration (env overrides defaults):')
const mergedResult = mergedConfig.build()
match(
  (config) => {
    console.log('   App name (default):', config.get('app.name'))
    console.log('   App port (env override):', config.get('app.port'))
    console.log('   Debug (env override):', config.get('app.debug'))
    console.log('   Database name (default):', config.get('database.name'))
    console.log('   Database host (env override):', config.get('database.host'))
    console.log('   Cache backend (env override):', config.get('cache.backend'))
  },
  (error) => console.error('   Failed:', error.message),
  mergedResult
)

Deno.jupyter.html`
<div style="background: #e3f2fd; padding: 15px; border-radius: 8px; margin: 15px 0;">
  <h4 style="color: #1565c0; margin: 0 0 10px 0;">🔄 Configuration Precedence</h4>
  <p style="margin: 0; color: #1565c0;">
    Configuration sources are merged <strong>left-to-right</strong>, with later sources overriding earlier ones:
    <br><code>defaults.merge(envVars).merge(cliArgs)</code>
    <br>This gives you predictable override behavior in production.
  </p>
</div>
`

## 2. Configuration Validation with Schemas

Raw configuration is just strings and objects. For production apps, you need validation and type safety:

In [ ]:
console.log('\n=== Configuration Validation Demo ===\n')

// Let's create a comprehensive configuration system
function createAppConfig(): Result<ValidatedConfig, QiError> {
  // Start with sensible defaults
  const defaults = ConfigBuilder.fromObject({
    app: {
      name: 'QiCore App',
      port: 3000,
      debug: false,
      logLevel: 'info'
    },
    database: {
      host: 'localhost',
      port: 5432,
      name: 'qiapp_dev',
      ssl: false
    },
    cache: {
      backend: 'memory',
      maxSize: 1000,
      ttl: 300  // 5 minutes
    },
    features: {
      enableMetrics: false,
      enableTracing: false
    }
  })

  // Add environment overrides
  const withEnv = defaults.merge(ConfigBuilder.fromEnv('QIAPP_'))

  // Build the merged configuration
  return withEnv.build()
}

// Create and test the configuration
const configResult = createAppConfig()

match(
  (config) => {
    console.log('✅ Configuration loaded successfully!')
    console.log('\nApplication Settings:')
    console.log(`  Name: ${config.get('app.name')}`)
    console.log(`  Port: ${config.get('app.port')}`)
    console.log(`  Debug: ${config.get('app.debug')}`)
    console.log(`  Log Level: ${config.get('app.logLevel')}`)
    
    console.log('\nDatabase Settings:')
    console.log(`  Host: ${config.get('database.host')}`)
    console.log(`  Port: ${config.get('database.port')}`)
    console.log(`  Database: ${config.get('database.name')}`)
    console.log(`  SSL: ${config.get('database.ssl')}`)
    
    console.log('\nCache Settings:')
    console.log(`  Backend: ${config.get('cache.backend')}`)
    console.log(`  Max Size: ${config.get('cache.maxSize')}`)
    console.log(`  TTL: ${config.get('cache.ttl')} seconds`)
    
    // Demonstrate safe access with fallbacks
    console.log('\nSafe Access Examples:')
    console.log(`  Timeout (with default): ${config.getOr('app.timeout', 30000)}ms`)
    console.log(`  Max connections (with default): ${config.getOr('database.maxConnections', 10)}`)
    
    return config
  },
  (error) => {
    console.error('❌ Configuration failed:')
    console.error(`   ${error.message}`)
    if (error.context?.source) {
      console.error(`   Source: ${error.context.source}`)
    }
    return null
  },
  configResult
)

// Demonstrate error handling with invalid configuration
console.log('\n--- Testing Invalid Configuration ---')

// Set an invalid port to see validation in action
Deno.env.set('QIAPP_APP_PORT', 'not-a-number')
const invalidConfig = createAppConfig()

match(
  (config) => console.log('Unexpectedly succeeded!'),
  (error) => {
    console.log(`❌ Configuration validation caught invalid port: ${error.message}`)
    
    // In production, you'd handle this error appropriately
    switch (error.category) {
      case 'VALIDATION':
        console.log('   → Fix the configuration and restart')
        break
      case 'CONFIGURATION':
        console.log('   → Check config file paths and permissions')
        break
      default:
        console.log(`   → Handle ${error.category} error appropriately`)
    }
  },
  invalidConfig
)

// Reset the environment variable
Deno.env.set('QIAPP_APP_PORT', '8080')

## 3. Structured Logging with Context

Logging is essential for observability. QiCore's logger provides structured logging with context accumulation:

In [ ]:
console.log('\n=== Structured Logging Demo ===\n')

// Create a logger with configuration
const loggerResult = createLogger({
  level: 'info',
  pretty: true,  // Pretty printing for this demo
  name: 'QiCore-Demo'
})

await match(
  async (logger) => {
    console.log('✅ Logger created successfully!')
    
    // Basic logging at different levels
    logger.debug('Debug message (not shown - level is info)', undefined, { debugInfo: 'hidden' })
    logger.info('Application started', undefined, { 
      version: '1.0.0',
      environment: 'development',
      timestamp: new Date().toISOString()
    })
    
    logger.warn('This is a warning', undefined, {
      component: 'demo',
      action: 'test-logging'
    })
    
    // Logging with QiError objects
    const sampleError = validationError('Sample validation failure')
    logger.error('Validation failed', sampleError, {
      userId: 123,
      operation: 'user-registration',
      attempt: 2
    })
    
    console.log('\n--- Child Logger with Inherited Context ---')
    
    // Create a child logger with inherited context
    const requestLogger = logger.child({ 
      requestId: 'req-123',
      userId: 456,
      operation: 'user-profile-update'
    })
    
    // All messages from this child logger will include the context
    requestLogger.info('Processing user request')
    requestLogger.info('Validating input', undefined, { field: 'email' })
    requestLogger.info('Updating database', undefined, { table: 'users', id: 456 })
    requestLogger.info('Request completed', undefined, { 
      duration: 156,
      success: true 
    })
    
    console.log('\n--- Another Child Logger (Different Context) ---')
    
    const authLogger = logger.child({ 
      component: 'authentication',
      sessionId: 'sess-abc-def'
    })
    
    authLogger.info('User login attempt', undefined, { username: 'alice' })
    authLogger.warn('Invalid password attempt', undefined, { 
      username: 'alice',
      attempts: 2,
      maxAttempts: 5 
    })
    authLogger.info('Login successful', undefined, { 
      username: 'alice',
      loginDuration: 87 
    })
    
    console.log('\n--- Logging Complex Objects ---')
    
    // Log complex structured data
    const userData = {
      id: 789,
      name: 'Bob Johnson',
      email: 'bob@example.com',
      preferences: {
        theme: 'dark',
        notifications: true,
        language: 'en'
      },
      metadata: {
        createdAt: '2024-01-15T10:30:00Z',
        lastLogin: '2024-01-20T14:22:15Z'
      }
    }
    
    logger.info('User data retrieved', undefined, {
      operation: 'get-user',
      user: userData,
      cacheHit: true,
      queryTime: 23
    })
    
    return logger
  },
  async (error) => {
    console.error('❌ Logger creation failed:', error.message)
    return null
  },
  loggerResult
)

Deno.jupyter.html`
<div style="background: #fff3cd; padding: 15px; border-radius: 8px; margin: 15px 0;">
  <h4 style="color: #856404; margin: 0 0 10px 0;">📝 Structured Logging Benefits</h4>
  <ul style="margin: 10px 0; padding-left: 20px; color: #856404;">
    <li><strong>Context Inheritance</strong> - Child loggers automatically include parent context</li>
    <li><strong>JSON Structure</strong> - Easy parsing by log aggregation tools</li>
    <li><strong>Level Filtering</strong> - Control verbosity in different environments</li>
    <li><strong>Error Integration</strong> - QiError objects logged with full context</li>
    <li><strong>Performance</strong> - Built on Pino for high-performance logging</li>
  </ul>
</div>
`

## 4. Caching - Memory and Redis Backends

Caching is crucial for performance. QiCore provides a unified interface for both memory and Redis caching:

In [ ]:
console.log('\n=== Caching Demo ===\n')

// First, let's create a memory cache (no external dependencies)
const memoryCacheResult = createCache({
  backend: 'memory',
  maxSize: 100,      // Maximum number of items
  defaultTtl: 60     // Default TTL in seconds
})

await match(
  async (cache) => {
    console.log('✅ Memory cache created successfully!')
    
    // Basic cache operations
    console.log('\n--- Basic Cache Operations ---')
    
    // Set some values
    await match(
      async () => console.log('✅ Cached user:123'),
      async (error) => console.error('❌ Cache set failed:', error.message),
      await cache.set('user:123', { name: 'Alice', email: 'alice@example.com' })
    )
    
    await match(
      async () => console.log('✅ Cached user:456'),
      async (error) => console.error('❌ Cache set failed:', error.message),
      await cache.set('user:456', { name: 'Bob', email: 'bob@example.com' })
    )
    
    await match(
      async () => console.log('✅ Cached config:app'),
      async (error) => console.error('❌ Cache set failed:', error.message),
      await cache.set('config:app', { port: 8080, debug: true }, 30) // Custom TTL
    )
    
    // Get values back
    console.log('\n--- Cache Retrieval ---')
    
    const user123Result = await cache.get('user:123')
    await match(
      async (user) => console.log('✅ Retrieved user:123:', user),
      async (error) => console.log('❌ Cache miss or error:', error.message),
      user123Result
    )
    
    const configResult = await cache.get('config:app')
    await match(
      async (config) => console.log('✅ Retrieved config:app:', config),
      async (error) => console.log('❌ Cache miss or error:', error.message),
      configResult
    )
    
    // Try to get a key that doesn't exist
    const missingResult = await cache.get('user:999')
    await match(
      async (value) => console.log('Unexpected success:', value),
      async (error) => console.log('✅ Expected cache miss for user:999:', error.message),
      missingResult
    )
    
    console.log('\n--- Batch Operations ---')
    
    // Set multiple values at once
    const batchSetData = {
      'product:1': { name: 'Laptop', price: 999 },
      'product:2': { name: 'Mouse', price: 25 },
      'product:3': { name: 'Keyboard', price: 75 }
    }
    
    await match(
      async () => console.log('✅ Batch set completed'),
      async (error) => console.error('❌ Batch set failed:', error.message),
      await cache.mset(batchSetData)
    )
    
    // Get multiple values at once
    const batchGetResult = await cache.mget(['product:1', 'product:2', 'product:999'])
    await match(
      async (products) => {
        console.log('✅ Batch get results:')
        Object.entries(products).forEach(([key, value]) => {
          console.log(`   ${key}:`, value || 'not found')
        })
      },
      async (error) => console.error('❌ Batch get failed:', error.message),
      batchGetResult
    )
    
    console.log('\n--- Cache Statistics ---')
    
    // Get cache statistics
    const stats = cache.getStats()
    console.log('Cache Statistics:')
    console.log('  Size:', stats.size)
    console.log('  Hit Rate:', `${(stats.hitRate * 100).toFixed(1)}%`)
    console.log('  Hits:', stats.hits)
    console.log('  Misses:', stats.misses)
    
    console.log('\n--- Cache Key Management ---')
    
    // List all keys (memory cache feature)
    const keysResult = await cache.keys('user:*')
    await match(
      async (keys) => console.log('✅ User keys:', keys),
      async (error) => console.log('❌ Failed to list keys:', error.message),
      keysResult
    )
    
    // Delete a key
    await match(
      async () => console.log('✅ Deleted user:456'),
      async (error) => console.error('❌ Delete failed:', error.message),
      await cache.del('user:456')
    )
    
    // Verify deletion
    const deletedResult = await cache.get('user:456')
    await match(
      async (value) => console.log('Unexpected - still found:', value),
      async (error) => console.log('✅ Confirmed deletion - key not found'),
      deletedResult
    )
    
    return cache
  },
  async (error) => {
    console.error('❌ Memory cache creation failed:', error.message)
    return null
  },
  memoryCacheResult
)

// Simulate Redis cache (would require actual Redis connection in production)
console.log('\n--- Redis Cache (Simulated) ---')
console.log('In production, you would configure Redis cache like this:')
console.log(`
const redisCacheResult = createCache({
  backend: 'redis',
  redis: {
    host: 'localhost',
    port: 6379,
    password: 'optional-password'
  },
  defaultTtl: 300  // 5 minutes
})
`)

Deno.jupyter.html`
<div style="background: #f8f9fa; padding: 15px; border-left: 4px solid #17a2b8; margin: 15px 0;">
  <h4 style="color: #117a8b; margin: 0 0 10px 0;">⚡ Cache Performance Tips</h4>
  <ul style="margin: 10px 0; padding-left: 20px; color: #117a8b;">
    <li><strong>Memory Cache</strong> - Fastest, but limited by process memory</li>
    <li><strong>Redis Cache</strong> - Shared across processes, persistent, scalable</li>
    <li><strong>TTL Strategy</strong> - Short TTL for volatile data, long for stable data</li>
    <li><strong>Key Patterns</strong> - Use consistent naming like "user:id", "config:app"</li>
    <li><strong>Batch Operations</strong> - More efficient for multiple keys</li>
  </ul>
</div>
`

## 5. Real-World Integration - E-commerce Product Service

Let's build a realistic service that uses all three infrastructure components together:

In [ ]:
console.log('\n=== E-commerce Product Service Demo ===\n')

// Domain types
interface Product {
  id: number
  name: string
  price: number
  category: string
  inStock: boolean
  description?: string
  lastUpdated: string
}

interface ProductService {
  config: Config
  logger: Logger
  cache: Cache
}

// Create the service with all dependencies
async function createProductService(): Promise<Result<ProductService, QiError>> {
  console.log('🏗️ Initializing Product Service...')
  
  // Step 1: Load configuration
  const serviceConfig = ConfigBuilder.fromObject({
    service: {
      name: 'ProductService',
      version: '1.2.3',
      timeout: 5000
    },
    cache: {
      enabled: true,
      ttl: 300,        // 5 minutes
      maxSize: 1000
    },
    logging: {
      level: 'info',
      pretty: true
    },
    database: {
      host: 'localhost',
      timeout: 3000
    }
  }).merge(ConfigBuilder.fromEnv('PRODUCT_'))
  
  const configResult = serviceConfig.build()
  
  return match(
    async (config) => {
      console.log('✅ Configuration loaded')
      
      // Step 2: Create logger with service context
      const loggerResult = createLogger({
        level: config.getOr('logging.level', 'info'),
        pretty: config.getOr('logging.pretty', false),
        name: config.get('service.name')
      })
      
      return match(
        async (baseLogger) => {
          const logger = baseLogger.child({
            service: config.get('service.name'),
            version: config.get('service.version')
          })
          
          logger.info('Logger initialized')
          
          // Step 3: Create cache if enabled
          const cacheEnabled = config.getOr('cache.enabled', false)
          
          if (cacheEnabled) {
            const cacheResult = createCache({
              backend: 'memory',
              maxSize: config.getOr('cache.maxSize', 100),
              defaultTtl: config.getOr('cache.ttl', 300)
            })
            
            return match(
              async (cache) => {
                logger.info('Cache initialized', undefined, {
                  backend: 'memory',
                  maxSize: config.get('cache.maxSize'),
                  ttl: config.get('cache.ttl')
                })
                
                return success({ config, logger, cache })
              },
              async (error) => failure(error),
              cacheResult
            )
          } else {
            logger.warn('Cache disabled - performance may be impacted')
            // Create a null cache that always misses
            const nullCache = {
              async get() { return failure(systemError('Cache disabled')) },
              async set() { return success(undefined) },
              async del() { return success(undefined) },
              async mget() { return success({}) },
              async mset() { return success(undefined) },
              async keys() { return success([]) },
              getStats() { return { size: 0, hits: 0, misses: 0, hitRate: 0 } }
            } as Cache
            
            return success({ config, logger, cache: nullCache })
          }
        },
        async (error) => failure(error),
        loggerResult
      )
    },
    async (error) => failure(error),
    configResult
  )
}

// Mock database operations (in production, these would hit a real database)
const mockDatabase = {
  products: new Map<number, Product>([
    [1, { id: 1, name: 'Gaming Laptop', price: 1299, category: 'Electronics', inStock: true, description: 'High-performance gaming laptop', lastUpdated: '2024-01-15T10:00:00Z' }],
    [2, { id: 2, name: 'Wireless Mouse', price: 49, category: 'Accessories', inStock: true, description: 'Ergonomic wireless mouse', lastUpdated: '2024-01-14T15:30:00Z' }],
    [3, { id: 3, name: 'Mechanical Keyboard', price: 129, category: 'Accessories', inStock: false, description: 'RGB mechanical keyboard', lastUpdated: '2024-01-13T09:15:00Z' }],
    [4, { id: 4, name: '4K Monitor', price: 399, category: 'Electronics', inStock: true, description: '27-inch 4K display', lastUpdated: '2024-01-16T14:20:00Z' }]
  ])
}

// Product service methods
async function getProduct(service: ProductService, id: number): Promise<Result<Product, QiError>> {
  const { config, logger, cache } = service
  const requestId = `req_${Date.now()}_${Math.random().toString(36).substr(2, 6)}`
  
  const requestLogger = logger.child({ requestId, operation: 'getProduct', productId: id })
  
  requestLogger.info('Product request started')
  
  // Try cache first
  const cacheKey = `product:${id}`
  const cacheResult = await cache.get<Product>(cacheKey)
  
  return match(
    async (cachedProduct) => {
      requestLogger.info('Cache hit', undefined, { cacheKey })
      return success(cachedProduct)
    },
    async (cacheError) => {
      requestLogger.info('Cache miss, querying database', undefined, { cacheKey })
      
      // Simulate database query delay
      await new Promise(resolve => setTimeout(resolve, 50))
      
      const product = mockDatabase.products.get(id)
      if (!product) {
        const error = validationError(`Product ${id} not found`)
        requestLogger.warn('Product not found', error, { productId: id })
        return failure(error)
      }
      
      // Cache the result
      await cache.set(cacheKey, product, config.getOr('cache.ttl', 300))
      
      requestLogger.info('Product retrieved and cached', undefined, {
        productId: id,
        productName: product.name,
        cacheKey,
        dbQueryTime: 50
      })
      
      return success(product)
    },
    cacheResult
  )
}

async function searchProducts(service: ProductService, category: string): Promise<Result<Product[], QiError>> {
  const { config, logger, cache } = service
  const requestId = `req_${Date.now()}_${Math.random().toString(36).substr(2, 6)}`
  
  const requestLogger = logger.child({ requestId, operation: 'searchProducts', category })
  
  requestLogger.info('Product search started')
  
  // Try cache first
  const cacheKey = `search:category:${category}`
  const cacheResult = await cache.get<Product[]>(cacheKey)
  
  return match(
    async (cachedResults) => {
      requestLogger.info('Search cache hit', undefined, { 
        cacheKey, 
        resultCount: cachedResults.length 
      })
      return success(cachedResults)
    },
    async (cacheError) => {
      requestLogger.info('Search cache miss, querying database', undefined, { cacheKey })
      
      // Simulate database query delay
      await new Promise(resolve => setTimeout(resolve, 100))
      
      // Filter products by category
      const results = Array.from(mockDatabase.products.values())
        .filter(product => product.category.toLowerCase() === category.toLowerCase())
      
      // Cache the results (shorter TTL for search results)
      const searchTtl = Math.floor(config.getOr('cache.ttl', 300) / 2)
      await cache.set(cacheKey, results, searchTtl)
      
      requestLogger.info('Search completed and cached', undefined, {
        category,
        resultCount: results.length,
        cacheKey,
        cacheTtl: searchTtl,
        dbQueryTime: 100
      })
      
      return success(results)
    },
    cacheResult
  )
}

// Test the complete service
const serviceResult = await createProductService()

await match(
  async (service) => {
    console.log('🎉 Product Service initialized successfully!\n')
    
    // Test individual product retrieval
    console.log('--- Testing Product Retrieval ---')
    
    for (const productId of [1, 2, 999]) {
      const productResult = await getProduct(service, productId)
      await match(
        async (product) => {
          console.log(`✅ Product ${product.id}: ${product.name} - $${product.price}`)
          console.log(`   In Stock: ${product.inStock}, Category: ${product.category}`)
        },
        async (error) => {
          console.log(`❌ Product ${productId}: ${error.message}`)
        },
        productResult
      )
    }
    
    console.log('\n--- Testing Cache Performance (Retrieving Same Product) ---')
    
    // Get the same product twice to test cache hit
    console.log('First request (should hit database):')
    await getProduct(service, 1)
    
    console.log('\nSecond request (should hit cache):')
    await getProduct(service, 1)
    
    console.log('\n--- Testing Product Search ---')
    
    for (const category of ['Electronics', 'Accessories', 'Clothing']) {
      const searchResult = await searchProducts(service, category)
      await match(
        async (products) => {
          console.log(`✅ Found ${products.length} products in ${category}:`)
          products.forEach(product => {
            console.log(`   - ${product.name} ($${product.price}) - ${product.inStock ? 'In Stock' : 'Out of Stock'}`)
          })
        },
        async (error) => {
          console.log(`❌ Search failed for ${category}: ${error.message}`)
        },
        searchResult
      )
    }
    
    console.log('\n--- Cache Statistics ---')
    const stats = service.cache.getStats()
    console.log(`Cache Performance:`)
    console.log(`  Total Operations: ${stats.hits + stats.misses}`)
    console.log(`  Cache Hits: ${stats.hits}`)
    console.log(`  Cache Misses: ${stats.misses}`)
    console.log(`  Hit Rate: ${(stats.hitRate * 100).toFixed(1)}%`)
    console.log(`  Cache Size: ${stats.size} items`)
    
    service.logger.info('Demo completed successfully', undefined, {
      totalRequests: stats.hits + stats.misses,
      cacheHitRate: stats.hitRate,
      cacheSize: stats.size
    })
  },
  async (error) => {
    console.error('❌ Service initialization failed:', error.message)
  },
  serviceResult
)

Deno.jupyter.html`
<div style="background: linear-gradient(135deg, #28a745 0%, #20c997 100%); color: white; padding: 20px; border-radius: 10px; margin: 20px 0; text-align: center;">
  <h3 style="margin: 0 0 15px 0;">🏆 Complete Integration Success!</h3>
  <p style="margin: 0; font-size: 16px;">
    You've seen configuration, logging, and caching work together in a realistic service architecture!
  </p>
</div>
`

## 🎯 Key Takeaways

After completing this tutorial, you now understand:

### ✅ **Configuration Management**
- `ConfigBuilder` supports multiple sources (objects, files, environment)
- **Merge precedence** - later sources override earlier ones
- **Validation** with schemas ensures type safety
- **Result<T> patterns** for safe configuration loading

### ✅ **Structured Logging**
- **Context inheritance** through child loggers
- **Structured data** with JSON output for log aggregation
- **QiError integration** for consistent error logging
- **Performance** with Pino backend

### ✅ **Unified Caching**
- **Memory cache** for single-process applications
- **Redis cache** for distributed applications (shown conceptually)
- **Batch operations** for better performance
- **TTL management** and statistics tracking

### ✅ **Service Integration**
- **Dependency injection** with Result<T> error handling
- **Request tracing** with logger context
- **Cache-aside pattern** for database optimization
- **Configuration-driven** service behavior

### ✅ **Production Patterns**
- **Error categorization** for appropriate handling
- **Performance monitoring** with cache statistics
- **Observability** through structured logging
- **Graceful degradation** when services are unavailable

## 🚀 Next Steps

Now that you understand the core infrastructure services, you're ready to learn about the message-driven architecture:

### 📖 **Continue Learning**
- **[03-qi-amsg.ipynb](./03-qi-amsg.ipynb)** - Async message queue system and h2A patterns
- **[04-qi-cli.ipynb](./04-qi-cli.ipynb)** - CLI framework with state management
- **[01-qi-base.ipynb](./01-qi-base.ipynb)** - Review Result<T> fundamentals if needed

### 🛠️ **Try It Yourself**
- Explore the working examples in `typescript/app/config-example/`
- Run `bun run dev` to see configuration and logging in action
- Experiment with different cache backends (Redis in production)

### 📚 **Dive Deeper**
- Read the [Core API Documentation](../api/core/README.md)
- Study the architectural patterns in `docs/core/README.md`
- Learn about event-driven patterns with logger events

**Remember**: These infrastructure services form the foundation for reliable, observable, and performant applications. Every production service should have proper configuration, logging, and caching!

In [ ]:
// Final challenge: Build your own service using all three components!
// Here's a template - adapt it for your use case

async function createMyService() {
  // Step 1: Define your configuration
  const config = ConfigBuilder.fromObject({
    myService: {
      name: 'MyCustomService',
      timeout: 5000
    }
    // Add your config properties here
  }).merge(ConfigBuilder.fromEnv('MY_'))
  
  const configResult = config.build()
  
  return match(
    async (cfg) => {
      // Step 2: Create logger
      const loggerResult = createLogger({
        name: cfg.get('myService.name'),
        level: 'info'
      })
      
      return match(
        async (logger) => {
          logger.info('Service initializing')
          
          // Step 3: Create cache
          const cacheResult = createCache({ backend: 'memory' })
          
          return match(
            async (cache) => {
              logger.info('Service ready')
              return success({ config: cfg, logger, cache })
            },
            async (error) => failure(error),
            cacheResult
          )
        },
        async (error) => failure(error),
        loggerResult
      )
    },
    async (error) => failure(error),
    configResult
  )
}

// Test your service
const myServiceResult = await createMyService()
match(
  (service) => {
    console.log('\n🎉 Your custom service is ready!')
    console.log('Service name:', service.config.get('myService.name'))
    service.logger.info('Custom service test completed')
  },
  (error) => {
    console.error('❌ Service creation failed:', error.message)
  },
  myServiceResult
)

Deno.jupyter.html`
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 20px; border-radius: 10px; margin: 20px 0; text-align: center;">
  <h3 style="margin: 0 0 15px 0;">🎉 QiCore Core Tutorial Complete!</h3>
  <p style="margin: 0; font-size: 16px;">
    You're now ready to build production-ready services with proper infrastructure!
  </p>
</div>
`